In [1]:
import os
import numpy as np
import pandas as pd
from pprint import pprint
import scipy.io as sio
import seaborn as sns
import mne
from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP

import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score, GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier



In [2]:
# Import the raw file using mne
raw = mne.io.read_raw_gdf('./BCICIV_2a_gdf/A01T.gdf', preload=True)
raw2 = mne.io.read_raw_gdf('./BCICIV_2a_gdf/A02T.gdf', preload=True)
raw3 = mne.io.read_raw_gdf('./BCICIV_2a_gdf/A03T.gdf', preload=True)

# set channel names to a standard
eegbci.standardize(raw) 

# Set up Arrays for the Classification Algorithm Scores

LH_RH = [] #Classification Algorithm Score Array for Left Hand vs Right Hand
LH_Tongue = [] #Classification Algorithm Score Array for Left Hand vs Tongue
LH_Foot = [] #Classification Algorithm Score Array for Left Hand vs Foot

Extracting EDF parameters from c:\Users\Darin Tsui\MotorImagery\BCICIV_2a_gdf\A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\contextlib.py:126: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Extracting EDF parameters from c:\Users\Darin Tsui\MotorImagery\BCICIV_2a_gdf\A02T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 677168  =      0.000 ...  2708.672 secs...


C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\contextlib.py:126: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Extracting EDF parameters from c:\Users\Darin Tsui\MotorImagery\BCICIV_2a_gdf\A03T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 660529  =      0.000 ...  2642.116 secs...


C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\contextlib.py:126: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


In [3]:
raw.ch_names

['EEG-Fz',
 'EEG-0',
 'EEG-1',
 'EEG-2',
 'EEG-3',
 'EEG-4',
 'EEG-5',
 'EEG-C3',
 'EEG-6',
 'EEG-Cz',
 'EEG-7',
 'EEG-C4',
 'EEG-8',
 'EEG-9',
 'EEG-10',
 'EEG-11',
 'EEG-12',
 'EEG-13',
 'EEG-14',
 'EEG-Pz',
 'EEG-15',
 'EEG-16',
 'EOG-LEFT',
 'EOG-CENTRAL',
 'EOG-RIGHT']

In [4]:
# Rename channels to work with teh montage standard
raw.rename_channels(lambda x: x.strip('EEG-'))
montage = mne.channels.make_standard_montage('standard_1005')

# Set our montage to our raw
raw.set_montage(montage, on_missing='ignore')

# Keep only Fz, 3, and 5 channels 
# excludeList = ['0', '2', '4', '5', 'C3', '6', 'Cz', '7', '8', '9', '10', '11', 
# '12', '13', '14', 'Pz', '15', '16', 'OG-LEFT', 'OG-CENTRAL', 'OG-RIGHT', 'OG-1', 'OG-2', 'OG-3', 'C4']
excludeList = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', 
'12', '13', '14', '15', '16', 'OG-LEFT', 'OG-CENTRAL', 'OG-RIGHT', 'OG-1', 'OG-2', 'OG-3', 'Fz', 'Pz']

# Setting up band pass filter which will filter from 8 to 30
raw.filter(8., 30., fir_design='firwin', skip_by_annotation='edge')

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.4s finished


<RawGDF | A01T.gdf, 25 x 672528 (2690.1 s), ~128.3 MB, data loaded>

In [5]:
# View Event Labels
mne.events_from_annotations(raw)

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


(array([[     0,      0,      5],
        [     0,      0,      3],
        [ 29683,      0,      5],
        ...,
        [670550,      0,      6],
        [670550,      0,      1],
        [671050,      0,      7]]),
 {'1023': 1,
  '1072': 2,
  '276': 3,
  '277': 4,
  '32766': 5,
  '768': 6,
  '769': 7,
  '770': 8,
  '771': 9,
  '772': 10})

In [6]:
# Define the different functions to seperate our classes
# They all seperate out from left hand
def filterEventsFoot(events):
    finalArray = []
    for event in events:
        if event[2] == 7 or event[2] == 9:
            finalArray.append(event)
    return finalArray

def filterEventsRight_Hand(events):
    finalArray = []
    for event in events:
        if event[2] == 7 or event[2] == 8:
            finalArray.append(event)
    return finalArray

def filterEventsTongue(events):
    finalArray = []
    for event in events:
        if event[2] == 7 or event[2] == 10:
            finalArray.append(event)
    return finalArray

In [7]:
events, _ = events_from_annotations(raw)

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


In [8]:
# grab the events and change it to be only left and right hand
events, _ = events_from_annotations(raw)
events = filterEventsRight_Hand(events)

# Excludelist was written in data Wrangling. defines the EEG channels we dont want
picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude=excludeList)

# set our events and epoch the data
event_id = dict(leftHand=7, rightHand=8)
epochs = Epochs(raw, events, event_id, 0, 6, proj=True, picks=picks, baseline=None, preload=True)
print(epochs.get_data().shape)

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
Not setting metadata
144 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 144 events and 1501 original time points ...
1 bad epochs dropped
(143, 3, 1501)


In [9]:
# Train data from between 1-4 seconds
epochs_train = epochs.copy().crop(tmin=1., tmax=4.)
labels = epochs.events[:, -1] - 2

In [10]:
# Split our testing data and training data with a .2 split
scores = []
epochs_data = epochs.get_data()
epochs_data_train = epochs_train.get_data()

# Randomly yielding indicies to split data into tests and train, and 5 interations of that.
cv = ShuffleSplit(5, test_size=0.2, random_state=42)
cv_split = cv.split(epochs_data_train)

In [11]:
# Using mne to declare our csp, in next code segment will fill it in with epoch data
csp = CSP(n_components=3, reg=None, log=True, norm_trace=False)
print(epochs.info)

<Info | 8 non-empty values
 bads: []
 ch_names: C3, Cz, C4
 chs: 3 EEG
 custom_ref_applied: False
 dig: 8 items (3 Cardinal, 5 EEG)
 highpass: 8.0 Hz
 lowpass: 30.0 Hz
 meas_date: 2005-01-17 12:00:00 UTC
 nchan: 3
 projs: []
 sfreq: 250.0 Hz
>


In [12]:
epochs_data.shape

(143, 3, 1501)

In [13]:
epochs

Number of events,143
Events,leftHand: 71rightHand: 72
Time range,0.000 – 6.000 sec
Baseline,off


In [14]:
epochs_data = epochs.get_data()

In [15]:
# plot CSP patterns estimated on full data for visualization using our data
csp.fit_transform(epochs_data, labels)

# csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)

Computing rank from data with rank=None
    Using tolerance 4.1e-06 (2.2e-16 eps * 3 dim * 6.2e+09  max singular value)
    Estimated rank (mag): 3
    MAG: rank 3 computed from 3 data channels with 0 projectors
Reducing data rank from 3 -> 3
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.9e-06 (2.2e-16 eps * 3 dim * 2.9e+09  max singular value)
    Estimated rank (mag): 3
    MAG: rank 3 computed from 3 data channels with 0 projectors
Reducing data rank from 3 -> 3
Estimating covariance using EMPIRICAL
Done.


array([[-2.11286276, -0.84093856, -0.69074258],
       [-1.8191899 , -0.7215149 , -0.48569596],
       [-2.19721384, -0.35735322, -0.63717884],
       [-1.98183444, -0.72720994, -0.50304597],
       [-2.04820076, -0.6696077 , -0.69223653],
       [-1.69912118, -0.35735504, -0.45076966],
       [-2.15462195, -0.8133044 , -0.96220394],
       [-1.98010359, -0.55908562, -0.70047421],
       [-2.22631284, -0.92953426, -0.87564122],
       [-1.9070611 , -0.70925205, -0.90112812],
       [-2.07291859, -0.78759122, -0.92415171],
       [-2.01880679, -0.76452055, -0.78651064],
       [-2.01511658, -0.61618196, -0.60499093],
       [-1.62846959, -0.49468722, -0.58020533],
       [-2.18342116, -0.91140488, -0.91963031],
       [-1.92654979, -0.58865485, -0.83865998],
       [-2.18144166, -0.7146613 , -0.73714766],
       [-2.04632342, -0.44284949, -0.57290788],
       [-1.96138331, -0.77153066, -0.6413652 ],
       [-1.51215721, -0.13407909, -0.36494694],
       [-2.02615517, -0.47358794, -0.422

In [16]:
# Assemble a classifier
lda = LinearDiscriminantAnalysis()

# SKlearn Pipeline with csp and lda, evaluating scores using Cross validation
clf = Pipeline([('CSP', csp), ('LDA', lda)])
scoresLDA = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1)

Computing rank from data with rank=None
    Using tolerance 1.2e-06 (2.2e-16 eps * 3 dim * 1.7e+09  max singular value)
    Estimated rank (mag): 3
    MAG: rank 3 computed from 3 data channels with 0 projectors
Reducing data rank from 3 -> 3
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.1e-06 (2.2e-16 eps * 3 dim * 1.7e+09  max singular value)
    Estimated rank (mag): 3
    MAG: rank 3 computed from 3 data channels with 0 projectors
Reducing data rank from 3 -> 3
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.2e-06 (2.2e-16 eps * 3 dim * 1.9e+09  max singular value)
    Estimated rank (mag): 3
    MAG: rank 3 computed from 3 data channels with 0 projectors
Reducing data rank from 3 -> 3
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1e-06 (2.2e-16 eps * 3 dim * 1.5e+09  max singular value)
    Estimated rank (m

In [17]:
# Printing the results our scores
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)
print("LDA accuracy: %f    |    Chance level: %f" % (np.mean(scoresLDA), class_balance))
LH_RH.append(np.mean(scoresLDA))

LDA accuracy: 0.765517    |    Chance level: 0.503497
